In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('habr.xlsx')

In [ ]:
df


,Unnamed: 0,Ссылка на объявление,Дата размещения,Название,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Описание,Работодатель,Ссылка на работодателя,Рейтинг,Офисы,Число сотрудников,Дата сбора данных
0,0,https://career.habr.com/vacancies/1000129552,01-04-2024 16:48:58,ИТ-редактор,40000.0,60000.0,₽,Копирайтер,Стажёр (Intern),Грамотность / Русский язык / Английский язык,NaN,Неполный рабочий день,1.0,~~~ О компании и команде\nМы готовим технологи...,Glyph media,https://glph.media,NaN,"Россия, Москва, пер. Токмаков, д.20/31, стр. 1",0,01-04-2024 20:35:58
1,1,https://career.habr.com/vacancies/1000138286,06-03-2024 10:31:40,Руководитель отдела разработки / CTO,300000.0,NaN,₽,Технический директор,NaN,Ведущий (Lead) / Linux / C# / .NET / WPF / Раз...,Пермь,Полный рабочий день,NaN,"~~~ О компании и команде\nМы – Macroscop, прод...",Macroscop,http://macroscop.com/,NaN,"Россия, Пермь, ш. Космонавтов, д. 111, корп. 27",4,01-04-2024 20:35:58
2,2,https://career.habr.com/vacancies/1000134771,20-03-2024 09:37:27,Программист 1С,NaN,NaN,NaN,Программист 1С,Средний (Middle),Разработка под 1С,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nАктивно развивающаяс...,ПартКом,http://www.part-kom.ru/,NaN,"Россия, Нижний Новгород, ул. Гаражная, д.9",8,01-04-2024 20:35:58
3,3,https://career.habr.com/vacancies/1000138674,27-03-2024 13:07:52,UX/UI дизайнер удаленно middle - senior,NaN,4000.0,$,UI/UX дизайнер,Средний (Middle),Figma Design / Adobe Photoshop / UI/UX дизайн ...,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nПривет! Приглашаем п...,IT Force,https://itforce.io/,NaN,"Россия, Москва, 1-й Красногвардейский пр.",2,01-04-2024 20:35:59
4,4,https://career.habr.com/vacancies/1000105132,31-03-2024 18:25:07,Разработчик ETL,NaN,NaN,NaN,Разработчик баз данных,Средний (Middle),SQL / PostgreSQL / ETL,"Москва, Санкт-Петербург",Полный рабочий день,1.0,~~~ О компании и команде\nBell Integrator – од...,Bell Integrator,http://www.bellintegrator.ru,4.68,"Беларусь, Минск, Беларусь, Минск, ул. Фабрициу...",288,01-04-2024 20:35:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,3333,https://career.habr.com/vacancies/1000128235,27-03-2024 15:27:14,Системный аналитик 1С,NaN,NaN,NaN,Системный аналитик,Старший (Senior),Разработка ТЗ / Agile / Автоматизация процессо...,"Воронеж, Краснодар, Ярославль",Полный рабочий день,NaN,~~~ О компании и команде\nПроект: Кредитный ко...,Совкомбанк Технологии,https://sovcombank.it/,4.69,NaN,205,01-04-2024 20:59:00
3334,3334,https://career.habr.com/vacancies/1000140564,26-03-2024 19:17:27,Middle Golang разработчик [Финтех продукт],NaN,NaN,NaN,Бэкенд разработчик,Средний (Middle),Redis / Golang / PostgreSQL / GraphQL / gRPC,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nМТС Digital – единый...,МТС,http://mts.ru,NaN,"Россия, Москва, пр-т Андропова, д. 18, корп. 9...",607,01-04-2024 20:58:58
3335,3335,https://career.habr.com/vacancies/1000133157,24-03-2024 14:27:33,Системный программист С,NaN,NaN,NaN,Десктоп разработчик,Средний (Middle),C / Linux / Python / Bash,NaN,Полный рабочий день,1.0,~~~ О компании и команде\nМы Группа Астра – од...,Группа Астра,https://astralinux.ru/,4.72,"Россия, Москва, Варшавское ш., д. 26",51,01-04-2024 20:59:01
3336,3336,https://career.habr.com/vacancies/1000140411,27-03-2024 15:50:28,QA Engineer (Mobile),NaN,NaN,NaN,Инженер по ручному тестированию,NaN,SOAP / SQL / Тестирование мобильных приложений...,Москва,NaN,1.0,~~~ О компании и команде\nBELL INTEGRATOR – кр...,Bell Integrator,http://www.bellintegrator.ru,4.68,"Беларусь, Минск, Беларусь, Минск, ул. Фабрициу...",288,01-04-2024 20:58:59


In [ ]:
df['Работодатель'].nunique()

631

In [ ]:
df.isna().sum()

Unnamed: 0                   0
Ссылка на объявление         0
Дата размещения              0
Название                     0
Зарплата от               2846
Зарплата до               2903
Валюта                    2693
Вакансия                     0
Стаж                       785
Требования                   0
Город                     1036
Тип занятости              300
Можно удаленно            1274
Описание                     0
Работодатель                 0
Ссылка на работодателя       0
Рейтинг                   1336
Офисы                      365
Число сотрудников            0
Дата сбора данных            0
dtype: int64

In [ ]:
df.columns

Index(['Unnamed: 0', 'Ссылка на объявление', 'Дата размещения', 'Название',
       'Зарплата от', 'Зарплата до', 'Валюта', 'Вакансия', 'Стаж',
       'Требования', 'Город', 'Тип занятости', 'Можно удаленно', 'Описание',
       'Работодатель', 'Ссылка на работодателя', 'Рейтинг', 'Офисы',
       'Число сотрудников', 'Дата сбора данных'],
      dtype='object')

In [ ]:
df = df[['Дата размещения', 'Зарплата от', 'Зарплата до', 'Валюта', 'Вакансия', 'Стаж', 'Требования', 'Город', 'Тип занятости', 'Можно удаленно', 'Работодатель']]
df

,Дата размещения,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Работодатель
0,01-04-2024 16:48:58,40000.0,60000.0,₽,Копирайтер,Стажёр (Intern),Грамотность / Русский язык / Английский язык,NaN,Неполный рабочий день,1.0,Glyph media
1,06-03-2024 10:31:40,300000.0,NaN,₽,Технический директор,NaN,Ведущий (Lead) / Linux / C# / .NET / WPF / Раз...,Пермь,Полный рабочий день,NaN,Macroscop
2,20-03-2024 09:37:27,NaN,NaN,NaN,Программист 1С,Средний (Middle),Разработка под 1С,NaN,Полный рабочий день,1.0,ПартКом
3,27-03-2024 13:07:52,NaN,4000.0,$,UI/UX дизайнер,Средний (Middle),Figma Design / Adobe Photoshop / UI/UX дизайн ...,NaN,Полный рабочий день,1.0,IT Force
4,31-03-2024 18:25:07,NaN,NaN,NaN,Разработчик баз данных,Средний (Middle),SQL / PostgreSQL / ETL,"Москва, Санкт-Петербург",Полный рабочий день,1.0,Bell Integrator
...,...,...,...,...,...,...,...,...,...,...,...
3333,27-03-2024 15:27:14,NaN,NaN,NaN,Системный аналитик,Старший (Senior),Разработка ТЗ / Agile / Автоматизация процессо...,"Воронеж, Краснодар, Ярославль",Полный рабочий день,NaN,Совкомбанк Технологии
3334,26-03-2024 19:17:27,NaN,NaN,NaN,Бэкенд разработчик,Средний (Middle),Redis / Golang / PostgreSQL / GraphQL / gRPC,NaN,Полный рабочий день,1.0,МТС
3335,24-03-2024 14:27:33,NaN,NaN,NaN,Десктоп разработчик,Средний (Middle),C / Linux / Python / Bash,NaN,Полный рабочий день,1.0,Группа Астра
3336,27-03-2024 15:50:28,NaN,NaN,NaN,Инженер по ручному тестированию,NaN,SOAP / SQL / Тестирование мобильных приложений...,Москва,NaN,1.0,Bell Integrator


In [ ]:
df['Требования'].value_counts()

Требования
SQL                                                                        36
Разработка под 1С                                                          33
Java                                                                       23
Информационная безопасность                                                23
Linux                                                                      21
                                                                           ..
Python / SQL / DWH / ETL / Apache Airflow                                   1
Анализ угроз / Тестирование на проникновение / OWASP                        1
Linux / Виртуализация / Администрирование Linux / Техническая поддержка     1
CI/CD / Тестирование мобильных приложений                                   1
Allure / Postman / Swagger / Ручное тестирование                            1
Name: count, Length: 2790, dtype: int64

**Удаляем строки, в которых одновременно отсутствует информация о зарплате в обоих столбцах**




In [ ]:
missing_data = df[df["Зарплата от"].isnull() & df["Зарплата до"].isnull()]

In [ ]:
df = df.drop(missing_data.index)
df

,Дата размещения,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Работодатель
0,01-04-2024 16:48:58,40000.0,60000.0,₽,Копирайтер,Стажёр (Intern),Грамотность / Русский язык / Английский язык,NaN,Неполный рабочий день,1.0,Glyph media
1,06-03-2024 10:31:40,300000.0,NaN,₽,Технический директор,NaN,Ведущий (Lead) / Linux / C# / .NET / WPF / Раз...,Пермь,Полный рабочий день,NaN,Macroscop
3,27-03-2024 13:07:52,NaN,4000.0,$,UI/UX дизайнер,Средний (Middle),Figma Design / Adobe Photoshop / UI/UX дизайн ...,NaN,Полный рабочий день,1.0,IT Force
8,27-03-2024 13:37:56,120000.0,180000.0,₽,Фулстек разработчик,Средний (Middle),HTML / React / Docker / PHP / CSS / Адаптивная...,NaN,Полный рабочий день,1.0,Новакабель
13,26-03-2024 19:06:04,100000.0,300000.0,₽,Менеджер продукта,Средний (Middle),Управление проектами / Управление людьми / Раз...,NaN,Полный рабочий день,1.0,Кошелёк.ру
...,...,...,...,...,...,...,...,...,...,...,...
3299,25-03-2024 16:58:00,170000.0,NaN,₽,ML разработчик,Средний (Middle),Python / PyTorch / TensorFlow / Apache Kafka /...,NaN,Полный рабочий день,1.0,Дром
3302,27-03-2024 20:29:37,NaN,100000.0,₽,Бэкенд разработчик,Средний (Middle),PHP / SQL / REST / Базы данных / Git,NaN,Полный рабочий день,1.0,presto.heads
3303,05-03-2024 13:13:05,NaN,300000.0,₽,DevOps-инженер,Средний (Middle),Linux / Nginx / PostgreSQL / RabbitMQ / GitLab...,Москва,Полный рабочий день,1.0,Информзащита
3320,01-04-2024 09:46:24,120000.0,NaN,₽,Бэкенд разработчик,Средний (Middle),Golang / FastAPI / SQLalchemy / gRPC / RabbitM...,Ставрополь,Полный рабочий день,NaN,ДругойКод


**Заменяем пропуски нулями и делаем столбец категориальным**

In [ ]:
df['Можно удаленно'].fillna(0, inplace=True)

In [ ]:
df['Можно удаленно'] = df['Можно удаленно'].astype('category')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 3326
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Дата размещения  645 non-null    object  
 1   Зарплата от      492 non-null    float64 
 2   Зарплата до      435 non-null    float64 
 3   Валюта           645 non-null    object  
 4   Вакансия         645 non-null    object  
 5   Стаж             543 non-null    object  
 6   Требования       645 non-null    object  
 7   Город            374 non-null    object  
 8   Тип занятости    610 non-null    object  
 9   Можно удаленно   645 non-null    category
 10  Работодатель     645 non-null    object  
dtypes: category(1), float64(2), object(8)
memory usage: 56.2+ KB
